# **N-Gram Language Models**

## Abstract

This notebook explores the n-gram language models and their implementation using the NLTK library.

## Table of Contents

>[N-Gram Language Models](#scrollTo=VB-1F_Zy6_Zk)

>>[Abstract](#scrollTo=evD9oHXk7A1-)

>>[Table of Contents](#scrollTo=kHLvGpIc7B9K)

>>[Language Modelling](#scrollTo=C4-uXyQb7kL6)

>>>[Language Models Evaluation](#scrollTo=cEZX9otO9IdZ)

>>[N-Grams](#scrollTo=UD_zm9ae-Mvu)

>>[N-Gram Language Models](#scrollTo=NWyAWHWM-Iwq)

>>>[Example: 4-gram Language Model](#scrollTo=KlVVhhRQAVkk)

>>>[Limitations of N-Gram Language Models](#scrollTo=3shPhibjB6n8)

>>[Implementation](#scrollTo=XqYbB-MYDZhz)

>>>[Import Libraries](#scrollTo=iF0ketsSDbjW)

>>>[Prepare the Dataset](#scrollTo=HxHvxJfIGfOy)

>>>[Everygrams Language Model](#scrollTo=O5yNzvaSia6f)

>>>[Using the Everygrams Trained Model](#scrollTo=eGrO-UwSkTci)

>>[References](#scrollTo=en2LHJOq7Eb-)



## Language Modelling

A **language model** allows us to predict the next most probable word, given a sequence of history words. More formally, given a sequence of words $w_1, ..., w_t$, a language model computes the conditional probability distribution of the next word $w_{t+1}$:

$P(w_{t+1} | w_1, ..., w_t)$

where $w_{t+1}$ can be any word in a vocabulary $V={w_1, ..., w_{|V|}}$. In this way, language modelling can be viewed as a classification task, as there’s a predefined number of possibilities, and the output is a probability distribution over them.

An **alternative** equivalent way of thinking about a language model is as a system which assigns a probability to a piece of text. Given a sequence of $T$ words, a language model assigns a probability $P(w_1, …, w_T)$ to the whole sequence, which can be broken down as follows:

$P(w_1, ..., w_T) = P(w_1) P(w_2 | w_1) P(w_3 | w_2, w_1) ...  P(w_T | w_{T-1}, ..., w_1)$

In practice, a language model gives the probability of a certain word sequence being “valid”. **Validity** in this context does not refer to grammatical validity. It means that it resembles how people speak (or, to be more precise, write) — which is what the language model learns. 


### Language Models Evaluation

The standard measurable evaluation metric for language models is perplexity. **Perplexity** is defined as the inverse probability of the corpus, according to the language model (the exponent is just for normalisation purposes, otherwise perplexity would get larger and larger as the corpus gets bigger).

$Perplexity = \prod_{t=1}^{T}(\frac{1}{P(w_{t+1}|w_t,...,w_1)})^\frac{1}{T}$

It can be shown that perplexity is equal to the exponential of the cross-entropy loss J():

$Perplexity=e^{J(\theta)}$


## N-Grams

By definition, an n-gram is a sequence of $n$ consecutive words.

- Unigrams:	  
  - “the”
- Bigrams:
  - “the students”
- Trigrams:
  - “the students opened”
- 4-grams:
  - “the students opened their”



## N-Gram Language Models

An n-gram model is a statistical model of language whose **core idea** is to collect statistics about how frequently different n-grams appear in a corpus of text, and use them to predict the next most probable next word.

N-grams language models make a simplifying **Markov assumption** that the next word $w_{t+1}$ depends solely on the preceding $(n-1)$ words. 

$P(x_{t+1}|x_t, x_{t-1}, ..., x_1) \approx P(x_{t+1}|x_t, x_{t-1}, ..., x_{t-n+2})$

This probability is computed as follows:

$P(x_{t+1}|x_t, x_{t-1}, ..., x_{t-n+2}) = \frac{P(x_{t+1}, x_t, x_{t-1}, ..., x_{t-n+2})}{P(x_t, x_{t-1}, ..., x_{t-n+2})} = \frac{Probability \; of \; the \; n-gram}{Probability \; of \; the \; (n-1)-gram}$

These n-gram probabilities are obtained by counting them in some large corpus of text (statistical approximation).

$\frac{Probability \; of \; the \; n-gram}{Probability \; of \; the \; (n-1)-gram} = \frac{Count(x_{t+1}, x_t, x_{t-1}, ..., x_{t-n+2})}{Count(x_t, x_{t-1}, ..., x_{t-n+2})}$

### Example: 4-gram Language Model

> As the proctor started the clock, the students opened their __________.

With a 4-gram language model, the next word depends solely on the previous 3 words:

> <font color="darkred">~~As the proctor started the clock, the~~</font> <font color="royalblue">students opened their</font> __________.

$P(w|students \; opened \; their)= \frac{count(students \; opened \; their \; w)}{count(students \; opened \; their)}$

Suppose that in the corpus:
- “students opened their” occurred 1000 times.
- “students opened their books” occurred 400 times: 
  $P(books|students \; opened \; their)=0.4$

- “students opened their exams” occurred 100 times.
  $P(exams|students \; opened \; their)=0.1$

Here we can notice the **problem with the Markov assumption**. By discarding the “proctor” context, which implies that the students are in an examination scenario, the prediction isn’t as good as it would’ve been if we kept the sentence’s context.

### Limitations of N-Gram Language Models

- As shown in the example above, **the Markov assumption problem** (it doesn’t take into account contexts that are farther than (n-1) words).

- **The sparsity problem**
  - If the n-gram never occurs in the corpus (numerator is zero). This is a problem because some words might be uncommon but they’re still valid and thus should have a non-zero probability.

    This problem is solved by adding a small smoothing factor to the count for every word in the vocabulary.

  - If the (n-1)-gram never occurs in the corpus (denominator is zero). In this case, we can’t even compute the probability distribution at all, for any word.

    A possible solution is to back-off to an (n-1)-gram LM.

- **Increasing n** 
  - Makes sparsity problems worse.
  - Increases the size of the model (to store the count for all the n-grams).

- **The lexical semantic problem** (it treats words as atomic units. Thus, there is no notion of similarity between words). This contributes to the sparsity problem.





## Implementation

This section uses NLTK's Language Model Module to implement a trigram language model.

### Import Libraries

In [ ]:
from nltk.lm.preprocessing import pad_both_ends, flatten, padded_everygram_pipeline
from nltk.util import ngrams, everygrams
from nltk.lm import MLE

import re

### Prepare the Dataset

This section prepares the corpus by putting it in the right format expected by trigram language model. This format consists of tokenizing the corpus into a list of sentences, each of which is to be tokenized into a list of words. At which point, a preprocessing pipeline gets a ready-for-training corpus and vocabulary as explained below.

In [ ]:
# A toy corpus is used
corpus = [
    "I really like doing nlp projects.",
    "Music is one of my passions.",
    "nlp is one of my passions.",
    "I love music."
]

To indicate that some words start and end sentences, special padding symbols are added to the sentence before splitting it into ngrams. This is done using NLTK's `pad_both_ends` function, which adds a `<s>` symbol at the start of the sentence and a `</s>` at its end. Additionally, to make the model more robust, we train it on unigrams (single words) as well as bigrams and trigrams. NLTK once again helpfully provides a function called `everygrams`. While not the most efficient, it is conceptually simple.

Finally, during training and evaluation, the model will rely on a vocabulary that defines which words are “known” to the model. To create this vocabulary we need to pad our sentences and then combine the sentences into one flat stream of words.

In [ ]:
def preprocess(text):

  # Lowercasing
  text = text.lower()

  # Keep only letters
  text = re.sub("[^a-z ]", "", text)

  # Splitting
  text = text.split(" ")

  return text

In [ ]:
def padded_everygrams_pipeline(corpus, max_n):
  """
  "I really like doing nlp projects.",
  "Music is one of my passions.",
  "nlp is one of my passions.",
  "I really love music."
  """
  # Keep only letters
  corpus = [re.sub("[^a-z ]", "", sentence.lower()) for sentence in corpus]

  # Split the corpus' sentences into words
  """
  ['I', 'really', 'like', 'doing', 'nlp', 'projects.']
  ['Music', 'is', 'one', 'of', 'my', 'passions.']
  ['nlp', 'is', 'one', 'of', 'my', 'passions.']
  ['I', 'really', 'love', 'music.']
  """
  split_corpus = [sentence.split(" ") for sentence in corpus]

  # Pad the corpus with special <s> </s> symbols
  """
  ['<s>', 'I', 'really', 'like', 'doing', 'nlp', 'projects.', '</s>']
  ['<s>', 'Music', 'is', 'one', 'of', 'my', 'passions.', '</s>']
  ['<s>', 'nlp', 'is', 'one', 'of', 'my', 'passions.', '</s>']
  ['<s>', 'I', 'really', 'love', 'music.', '</s>']
  """
  padded_corpus = [list(pad_both_ends(split_corpus[i], n=2))
  for i in range(len(split_corpus))]

  # Get unigrams, bigrams, ..., up to max_n-grams for the padded corpus
  """
  ('<s>',)
  ('<s>', 'I')
  ('<s>', 'I', 'really')
  ('I',)
  ('I', 'really')
  ('I', 'really', 'like')
  """
  padded_everygrams = [list(everygrams(padded_sentence, max_len=max_n))
  for padded_sentence in padded_corpus]

  # Get the vocabulary
  """
  ['<s>', 'I', 'really', 'like', 'doing', 'nlp', 'projects.', '</s>', 
  '<s>', 'Music', 'is', 'one', 'of', 'my', 'passions.', '</s>', 
  '<s>', 'nlp', 'is', 'one', 'of', 'my', 'passions.', '</s>', 
  '<s>', 'I', 'really', 'love', 'music.', '</s>']
  """
  vocabulary = set(flatten(padded_sentence
  for padded_sentence in padded_corpus))

  return padded_everygrams, vocabulary

### Everygrams Language Model

In [ ]:
# Trigrams
n = 3

# Apply the preprocessing pipeline to get the everygrams and the vocabulary
train, vocab = padded_everygrams_pipeline(corpus, 3)

# Instanciate a trigram language model
lm = MLE(3)

# Fit the model on the everygrams and the vocabulary
lm.fit(train, vocab)

### Using the Everygrams Trained Model

In [ ]:
for i in range(2):
  generated_text = " ".join(lm.generate(5, text_seed="i"))
  print(f"i {generated_text}")

i love music </s> nlp is
i really like doing nlp projects


## References

1. *Dan Jurafsky and James H. Martin. [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/) (3rd ed. draft).*

2. Stanford CS224n: Natural Language Processing with Deep Learning: [Language Models](https://web.stanford.edu/class/cs224n/index.html#coursework)

3. NLTK Documentation: [NLTK Language Modeling Module](https://www.nltk.org/api/nltk.lm.html)